In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks')

In [3]:
from __future__ import print_function

import argparse
import copy
import os
import sys
import numpy as np
from tqdm import tqdm, trange
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import os
import torchvision.utils as vutils
import seaborn as sns
import torch.nn.init as init
import pickle

from Lottery_Hypothesis import Lottery_Hypothesis

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'
print(device)

cuda


In [5]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])

traindataset = datasets.CIFAR10('../data', train=True, download=True,transform=transform)
valdataset = datasets.CIFAR10('../data', train=False, transform=transform)     

Files already downloaded and verified


In [0]:
train_loader = torch.utils.data.DataLoader(traindataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(valdataset, batch_size=32, shuffle=True)

In [0]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()

    self.cnn = nn.Sequential(
        nn.Conv2d(3, 6, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),

        nn.Conv2d(6, 12, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        
        nn.Conv2d(12, 24, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),

        nn.Conv2d(24, 48, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.fcc = nn.Sequential(
        nn.Linear(192, 96),
        nn.ReLU(),
        nn.Linear(96, 10)
    )

  def forward(self, x):
    feature = self.cnn(x)
    feature = feature.view(feature.size(0), -1)
    y_ = self.fcc(feature)
    return y_

In [0]:
net = Net()
net.to(device)
lottery = Lottery_Hypothesis(net, 0.01, 10, 5, train_loader, val_loader, val_loader, '/content/init_weight.pt', '/content/prune_weight.pt', 1e-3, nn.CrossEntropyLoss())

In [9]:
lottery.prune()

 10%|█         | 1/10 [01:13<10:58, 73.15s/it]

KeyboardInterrupt: ignored